In [ ]:
import numpy as np
import copy

In [ ]:
# Day 11
# This is most likely not the most performant way to do it, to much array copying...
with open("input11", "r") as fp:
    lines = fp.readlines()

field = []
for y, line in enumerate(lines):
    field.append([])
    for x, char in enumerate(line.strip()):
        field[y].append(".L".find(char))
field = np.array(field)

def get_part(x,y, kind="v1"):
    if kind == "v1":
        return field[max(y-1,0):y+2,max(x-1,0):x+2]
    else:
        dirs = [[-1,0],[1,0],[0,-1],[0,1],[1,1],[1,-1],[-1,1],[-1,-1]]
        result = np.zeros(9)
        result[8] = field[y,x]
        for ind, direction in enumerate(dirs):
            for i in range(1,10000):
                x2 = x+direction[0]*i
                y2 = y+direction[1]*i
                if x2 < 0 or y2 < 0 or x2 >= field.shape[1] or y2 >= field.shape[0]:
                    result[ind] = 0
                    break
                f = field[y2, x2]
                if f > 0:
                    result[ind] = f
                    break
        return np.array(result)


def born(x,y, kind="v1"):
    return np.max(get_part(x, y, kind)) < 2

def die(x,y, kind="v1"):
    part = get_part(x, y, kind)
    if kind == "v1":
        return np.sum(part==2) >= 5
    else:
        return np.sum(part==2) >= 6

def iteration(kind="v1"):
    global field
    new_field = field.copy()
    for y in range(field.shape[0]):
        for x in range(field.shape[1]):
            if field[y,x] == 0:
                continue
            if born(x,y,kind):
                new_field[y,x] = 2
            if die(x,y,kind):
                new_field[y,x] = 1
    field = new_field

original_field = field.copy()
last_field = field.copy()
for i in range(500):
    iteration("v1")
    if np.all(last_field == field):
        print(np.sum(field == 2))
        break
    last_field = field.copy()

field = original_field
last_field = field.copy()
for i in range(500):
    iteration("v2")
    if np.all(last_field == field):
        print(np.sum(field == 2))
        break
    last_field = field.copy()



In [ ]:
# Day 10
with open("input10", "r") as fp:
    lines = fp.readlines()
numbers = []
for line in lines:
    numbers.append(int(line))
numbers.sort()
numbers.append(numbers[-1]+3)
numbers.insert(0, 0)
diffs = np.diff(numbers)
print(len(diffs[diffs==1]) * len(diffs[diffs==3]))
# diff only consists of 1s and 3s. Every 3 "resets" the sequence (every sequence has to go through here)
# the number of 1s between 3s indicates how many different permutation between those two 3s are possible
# (e.g.) 3ABC3 -> 4 ways 3ABC3, 3AB3, 3AC3, 3BC3 (not that we already need one step to get to 3, so we basically can skip one 1)
num_1s = [len(n) -1 for n in np.split([3] + diffs.tolist(), np.where(diffs==3)[0]+1) if len(n) > 1]
mult = [0,1,2,4,7]
multipliers = [mult[i] for i in num_1s]
print(np.multiply.reduce(multipliers))

In [ ]:
# Day 9
with open("input9", "r") as fp:
    lines = fp.readlines()
numbers = []
for line in lines:
    numbers.append(int(line))

def check(number, number_list):
    for n1 in number_list:
        for n2 in number_list:
            if n1+n2 == number:
                return True
    return False

for i in range(25,len(numbers)):
    if not check(numbers[i], numbers[i-25:i]):
        print(numbers[i])
        invalid = numbers[i]
        break
# brute force :(
start = 0
end = 1
while True:
    sum_ = np.sum(numbers[start:end]) 
    if sum_ == invalid:
        print(np.max(numbers[start:end])+np.min(numbers[start:end]))
        break
    if sum_ > invalid:
        start += 1
        end = start + 1
    else:
        end += 1

In [ ]:
# Day 8
with open("input8", "r") as fp:
    lines = fp.readlines()

instructions = []
for line in lines:
    op = line.split()[0]
    num = int(line.split()[1])
    instructions.append([op, num])
acc = 0
visited = []
current = 0
while True:
    if current in visited:
        break
    visited.append(current)
    op = instructions[current]
    if op[0] == "acc":
        acc += op[1]
        current += 1
        continue
    if op[0] == "nop":
        current += 1
        continue
    if op[0] == "jmp":
        current += op[1]
        continue
print(acc)
# Brute force it
# We already know visited
import copy
wrong_visited = copy.copy(visited)
mod = -1
while True:
    instructions_new = copy.copy(instructions)
    # change next instruction from back (skipping acc)
    while True:
        if instructions[wrong_visited[mod]][0] == "nop":
            instructions_new[wrong_visited[mod]][0] = "jmp"
            break
        if instructions[wrong_visited[mod]][0] == "jmp":
            instructions_new[wrong_visited[mod]][0] = "nop"
            break
        mod -= 1
    mod -= 1
    # And test this for success
    acc = 0
    visited = []
    current = 0
    success = False
    while True:
        if current >= len(instructions_new):
            success = True
            break
        if current in visited:
            break
        visited.append(current)
        op = instructions_new[current]
        if op[0] == "acc":
            acc += op[1]
            current += 1
            continue
        if op[0] == "nop":
            current += 1
            continue
        if op[0] == "jmp":
            current += op[1]
            continue
    if success:
        break
print(acc)

In [ ]:
# Day 7
with open("input7", "r") as fp:
    lines = fp.readlines()
bags = {}
for line in lines:
    line = line.replace("bags", "").replace("bag", "").replace(".","")
    parts = line.split("contain")
    color = parts[0].strip()
    bags[color] = {}
    in_parts = parts[1].split(",")
    for part in in_parts:
        part = part.strip()
        if part == "no other":
            continue
        number = int(part.split()[0].strip())
        name = " ".join(part.split()[1:]).strip()
        bags[color][name] = number
def find_color(color):
    result = []
    for bag, in_bag in bags.items():
        if color in in_bag:
            result.append(bag)
            result.extend(find_color(bag))
    return result
print(len(np.unique(abc)))
def find_num(color):
    result = 1
    for bag, num in bags[color].items():
        result += find_num(bag)*num
    return result
print(find_num("shiny gold")-1)

In [ ]:
# Day 6
with open("input6", "r") as fp:
    lines = fp.readlines()
all_groups = []
current = []
for line in lines:
    if line.strip() == "":
        all_groups.append(current)
        current = []
    else:
        current.append(line.strip())
all_groups.append(current)
all_strings = ["".join(a) for a in all_groups]
all_unique = [len(np.unique(list(a))) for a in all_strings]
print(np.sum(all_unique))
all_every = []
for g in all_groups:
    if len(g) == 0:
        continue
    a = set(list(g[0]))
    if len(g) > 1:
        for b in g[1:]:
            a = a.intersection(set(list(b)))
    all_every.append(len(a))
print(np.sum(all_every))

In [ ]:
# Day 5
with open("input5", "r") as fp:
    lines = fp.readlines()
seat_nr = []
for line in lines:
    row = line[:7].replace("B","1").replace("F", "0")
    col = line[7:10].replace("R","1").replace("L", "0")
    row = int(row, base=2)
    col = int(col, base=2)
    id = row*8+col
    seat_nr.append([row, col, id])
print(np.max(np.array(seat_nr)[:,2]))
ids = [s[2] for s in seat_nr]
ids.sort()
diff = np.argmax(np.diff(ids))
print(np.sum(ids[diff:diff+2])//2)